In [2]:
%pip install ipython-ngql networkx pyvis
%load_ext ngql
# connect to NebulaGraph service
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula
# create a graph space(think of a Database Instance) named: llamaindex_nebula_property_graph
%ngql CREATE SPACE IF NOT EXISTS llamaindex_nebula_property_graph(vid_type=FIXED_STRING(256));


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[OK] Connection Pool Created


""


In [3]:
%ngql USE llamaindex_nebula_property_graph;

""


In [18]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-12-05 10:21:23--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.07s   

2024-12-05 10:21:23 (1.04 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [6]:
import nest_asyncio

nest_asyncio.apply()

In [7]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [8]:
print(type(documents))

<class 'list'>


In [9]:
import pandas as pd
from pathlib import Path
def load_dataframe(file_path):
    return pd.read_csv(file_path)

file_path = Path("LTL_3_cleaned.csv")  
df = load_dataframe(file_path)
null_values = df.isnull().sum().sum()
print(null_values)

0


In [10]:
row_counts = df.count(axis=1)

# Print the row counts
for index, count in enumerate(row_counts, start=1):
    print(f"Row {index} has {count} non-null entries")

Row 1 has 48 non-null entries
Row 2 has 48 non-null entries
Row 3 has 48 non-null entries
Row 4 has 48 non-null entries
Row 5 has 48 non-null entries
Row 6 has 48 non-null entries
Row 7 has 48 non-null entries
Row 8 has 48 non-null entries
Row 9 has 48 non-null entries
Row 10 has 48 non-null entries
Row 11 has 48 non-null entries
Row 12 has 48 non-null entries
Row 13 has 48 non-null entries
Row 14 has 48 non-null entries
Row 15 has 48 non-null entries
Row 16 has 48 non-null entries
Row 17 has 48 non-null entries
Row 18 has 48 non-null entries
Row 19 has 48 non-null entries
Row 20 has 48 non-null entries
Row 21 has 48 non-null entries
Row 22 has 48 non-null entries
Row 23 has 48 non-null entries
Row 24 has 48 non-null entries
Row 25 has 48 non-null entries
Row 26 has 48 non-null entries
Row 27 has 48 non-null entries
Row 28 has 48 non-null entries
Row 29 has 48 non-null entries
Row 30 has 48 non-null entries
Row 31 has 48 non-null entries
Row 32 has 48 non-null entries
Row 33 has 48 non

In [11]:
from llama_index.readers.file import CSVReader


reader = CSVReader()
documents = reader.load_data(file=file_path)


for doc in documents:
    print(doc.text) 
    print(doc.metadata) 


po_number, supplier_name, supplier_id, minority_supplier_certificate, sustainability_rating, financial_health_score, service_name, parent_category, category, description, catalog_id, route, equipment, total_weight_lbs, quantity, dimensions, distance_miles, base_rate_per_mile, fuel_surcharge, detention_rate_per_hour, liftgate_service_rate, total_estimated_cost, contract_utilised, payment_terms, currency, unit_of_measure, country, po_date, pickup_date, delivery_date, 
actual_receipt_date, special_instructions, additional_terms_and_conditions, additional_services, contract_id, contract_value, contract_term, contract_start_date, contract_end_date, authorized_by, ship_from, ship_from_address, ship_to, ship_to_address, department, cost_center, gl_account_id, requester
PO9902, C.H. Robinson, SUP0052, yes, 4.3, 4.6, LTL, Transportation Service, Ground freight, With the growth of ecommerce, less than truckload (LTL) cargo shipping demands more flexibility than ever. Manage spend without sacrifi

In [19]:
%pip install llama-index-embeddings-huggingface llama-index-llms-groq groq

933.31s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.llms.groq import Groq
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = Groq(model="llama3-70b-8192", api_key=os.getenv("GROQ_API_KEY"))
Settings.embed_model = HuggingFaceEmbedding(
    model_name="intfloat/multilingual-e5-large"
)
# Settings.embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [14]:
from llama_index.graph_stores.nebula import NebulaPropertyGraphStore

graph_store = NebulaPropertyGraphStore(
    space="llamaindex_nebula_property_graph", overwrite=True
)

In [15]:
from llama_index.core.vector_stores.simple import SimpleVectorStore

vec_store = SimpleVectorStore()

In [23]:
from llama_index.core.indices.property_graph import PropertyGraphIndex
from llama_index.core.storage.storage_context import StorageContext
from llama_index.llms.openai import OpenAI

index = PropertyGraphIndex.from_documents(
    documents,
    property_graph_store=graph_store,
    vector_store=vec_store,
    show_progress=True,
)

index.storage_context.vector_store.persist("./data/nebula_vec_store.json")

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting paths from text:  94%|█████████▍| 48/51 [09:04<00:34, 11.33s/it]


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jeb5zh2dfpbrtjmv7dvdwq24` on tokens per minute (TPM): Limit 6000, Used 6373, Requested 860. Please try again in 12.338s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}